# FIT5148 Assignment 2
#### Task C1, Part b

* Due: 24/05/2019
* Tutor: Paras Sitoula, Wednesday 12-2pm
<br>

| Student 1 | Student 2|
|-----------|----------|
| Hitesh Get | Samuel Campbell |

## 1. Import Necessary Modules

In [20]:
import os
import csv
import random
from time import sleep
import json
import geohash as gh
from kafka import KafkaProducer
import random
from datetime import datetime
from pprint import pprint

## 2. Set Working Directory

In [21]:
print("Please enter a valid path containing the climate and hotspot data")
os.chdir(input())

## 3. Read Satellite Data

In [22]:
with open('./hotspot_AQUA_streaming.csv') as file:
    hotspot_aqua_stream = list(csv.reader(file))

In [36]:
print("For your convenience, please find an example JSON-formatted satellite record below:")
pprint({
    hotspot_aqua_stream[0][0]: hotspot_aqua_stream[1][0],
    hotspot_aqua_stream[0][1]: hotspot_aqua_stream[1][1],
    hotspot_aqua_stream[0][2]: hotspot_aqua_stream[1][2],
    hotspot_aqua_stream[0][3]: hotspot_aqua_stream[1][3]
})

For your convenience, please find an example JSON-formatted satellite record below:
{'confidence': '51',
 'latitude': '-37.623',
 'longitude': '149.323',
 'surface_temperature_celcius': '38'}


## 4. Instantiate Producer

In [ ]:
def publish_message(producer_instance, topic_name, key, data):
    """
    This publishing function encodes the data according to the provided `key` (here json-formatted) 
    and character encoding format (here utf-8).
    
    Then, the data are broadcast to the server with the provided `topic_name`.
    """
    
    try:
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key = key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully.  ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print("Publish MEssage",str(ex))        

def connect_kafka_producer():
    """
    Instantiates a producer on localhost.
    """
    
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer = lambda x: json.dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
    # Set topic
    topic = 'climate'
    
    print('Publishing records..')
    # Connect the producer
    producer = connect_kafka_producer()
        
    # JSON-format each randomly-selected record and push it to the stream
    for i in random.sample(range(1, len(hotspot_aqua_stream)), len(hotspot_aqua_stream) - 1):
        hotspot_aqua = dict()
        hotspot_aqua["hotspot_aqua"] = {}
        hotspot_aqua["hotspot_aqua"]["latitude"] = float(hotspot_aqua_stream[i][0])
        hotspot_aqua["hotspot_aqua"]["longitude"] = float(hotspot_aqua_stream[i][1])
        hotspot_aqua["hotspot_aqua"]["confidence"] = float(hotspot_aqua_stream[i][2])
        hotspot_aqua["hotspot_aqua"]["surface_temperature_celcius"] = float(hotspot_aqua_stream[i][3])
        hotspot_aqua["Time_Stamp"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        hotspot_aqua["sender_id"] = "hotspot_streamer_aqua"
        hotspot_aqua['geohash'] = gh.encode(hotspot_aqua['hotspot_aqua']["latitude"], hotspot_aqua['hotspot_aqua']["longitude"], precision = 5)
        data = hotspot_aqua
        publish_message(producer, topic, 'jsondata', data)
        
        sleep(10)